# 7 BASIC STOCK DATA ANALYSIS METHODS

The objective of this assignment was to collect a dataset from one or more open web APIs of OWN choice, and use Python to pre-process and analyse the collected data.

The assignment was required to be implemented as a single IPython Notebook (not a script). The code was required be clearly documented, using comments and Markdown cells to explain the code and results.

The Tasks were completed are as follows:
- Extracted Data from 2 sources. The first source for stock code/ticker of NASDAQ-100 using BeautifulSoup. The second source is Yahoo Finance API using Python, parsed the data using  Pandas Library.
- Performed Pre-processing and quality checking steps, then analysed and summarised the cleaned dataset. Used various plots (Candlestick plot, Relative Strength Index, Moving Average Convergence Divergence) to represent the data using Dash Plotly Library.
- Based on the results, interpretations or insights about possible transaction decisions were made about the dataset.

# Import and clean data

In [1]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import chart_studio
import chart_studio.plotly as py
from datetime import datetime, timedelta

## Import list of stock codes for NASDAQ-100

In [ ]:
#Code to crawl data from Wikipedia website using BeautifulSoup
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
page=requests.get('https://en.wikipedia.org/wiki/NASDAQ-100',headers=headers)
soup=bs(page.content,"lxml")

In [ ]:
#Clean crawled data and sorted out company name and stock code/ticker
names=[]
tickers=[]
tabs=soup.find('table',id='constituents')
for row in tabs.findAll('tr'):
    cell=row.findAll('td')
    if len(cell)==2:
        names.append(cell[0].find(text=True))
        tickers.append(cell[1].find(text=True))

#Get crawled and cleaned data into data frame
df=pd.DataFrame(names,columns=['Company'])
df['Tickers']=tickers
df['Tickers']=df['Tickers'].str.replace('\n','')

In [ ]:
#Sample of crawled data after processed
df

## Import detail stock transaction data with stock code from Yahoo Finance

In [ ]:
#Function for extracting a company stock information from 2015-01-01 until today from Yahoo Finance
def info(ticker):
    today=date.today()
    info=pdr.get_data_yahoo(ticker,start='2015-01-01',end=today)
    return info

In [ ]:
#Test sample with Apple stock
info('AAPL') 

In [ ]:
#Crawl data for stock price of all company from NASDAQ-100 list above
ticker_lst=df['Tickers'].tolist()
today=date.today()
lst = []
for i in ticker_lst:
    info=pdr.get_data_yahoo(i,start='2015-01-01',end=today)
    info.columns = pd.MultiIndex.from_product([['%s'%i],info.columns])
    d=pd.DataFrame(info)
    lst.append(d)
    data=pd.concat(lst,axis=1) 
#     data.to_csv('/Users/ducng/Desktop/nasdag_finance/'+str(i)+'.csv') ##convert downloaded info to csv file.
data.head()

In [ ]:
# Another method for stock price crawling for a list.
# ticker_lst=df['Tickers'].tolist()
# today=date.today()
# data=pdr.get_data_yahoo(ticker_lst,start='2015-01-01',end=today)
# lst = [data['Adj Close']['AAPL'], data['Open']['AAPL'], data['Close']['AAPL'], 
#        data['High']['AAPL'], data['Low']['AAPL'],data['Volume']['AAPL']]
# test = pd.concat(lst,axis=1)
# test.columns = ['Adj Close', 'Open', 'Close', 'High', 'Low', 'Volume']
# test

Data has 6 main columns:
- Adj Close
- Open
- Close
- High
- Low
- Volume

# Technical analysis indicators - Sample on Apple stock AAPL

https://www.investopedia.com/top-7-technical-analysis-tools-4773275

## On-Balance Volume (OBV) 

Source: https://www.investopedia.com/terms/o/onbalancevolume.asp

On-balance volume (OBV) is a technical trading momentum indicator that uses volume flow to predict changes in stock price. Joseph Granville first developed the OBV metric in the 1963 book Granville's New Key to Stock Market Profits.

KEY TAKEAWAYS On-balance volume (OBV) is a technical indicator of momentum, using volume changes to make price predictions. OBV shows crowd sentiment that can predict a bullish or bearish outcome. Comparing relative action between price bars and OBV generates more actionable signals than the green or red volume histograms commonly found at the bottom of price charts.

Calculating OBV On-balance volume provides a running total of an asset's trading volume and indicates whether this volume is flowing in or out of a given security or currency pair. The OBV is a cumulative total of volume (positive and negative). There are three rules implemented when calculating the OBV. They are:

If today's closing price is higher than yesterday's closing price, then: Current OBV = Previous OBV + today's volume

If today's closing price is lower than yesterday's closing price, then: Current OBV = Previous OBV - today's volume

If today's closing price equals yesterday's closing price, then: Current OBV = Previous OBV

What Does On-Balance Volume Tell You? The theory behind OBV is based on the distinction between smart money – namely, institutional investors – and less sophisticated retail investors. As mutual funds and pension funds begin to buy into an issue that retail investors are selling, volume may increase even as the price remains relatively level. Eventually, volume drives the price upward. At that point, larger investors begin to sell, and smaller investors begin buying.

In [ ]:
#On-balance-Volume (OBV): the OBV will tend to follow the general trend of the market. 
#It will tend to increase in uptrends and decrease in downtrends.
#Compute obv for one company
def on_balance_volume(ticker, trend_periods):    # ticker = symbol of stock
    data = info(ticker)
#     data = df[ticker]
    data = data.reset_index(drop=False)
    obv = []
    for index, row in data.iterrows():
        if index > 0:
            last_obv = result.at[index - 1, 'obv']
            if row['Close'] > data.at[index - 1, 'Close']:
                current_obv = last_obv + row['Volume']
            elif row['Close'] < data.at[index - 1, 'Close']:
                current_obv = last_obv - row['Volume']
            else:
                current_obv = last_obv
        else:
            last_obv = 0
            current_obv = row['Volume']

        obv.append(current_obv)
        result = pd.DataFrame({'obv': obv})

    result['obv_ema' + str(trend_periods)] = result['obv'].ewm(ignore_na=False, min_periods=0, com=trend_periods, adjust=True).mean()
    data = pd.concat([data,result], axis=1)
    data = data.set_index('Date')
    return data

In [ ]:
#Testing on AAPL stock
on_balance_volume('AAPL', trend_periods = 21)

In [ ]:
#Code OBV graph for one stock code
def obv_fig(ticker, periods):
    data = on_balance_volume(ticker, periods)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.3)
    fig.add_trace(go.Candlestick(x = data.index,
                                 open = data['Open'],
                                 high = data['High'],
                                 low = data['Low'],
                                 close = data['Close'],
                                 name = 'Stock price'), 
                  row = 1, col = 1)

    avg_adj = data.Close.rolling(window=21, min_periods=1).mean()
    fig.add_trace(go.Scatter(x = data.index, 
                             y = avg_adj,
                             line = dict(color='blue', width=1, dash='dash'),
                             name = 'Close,' + str(periods) + '-Day EMA'),
                  row = 1, col = 1)

    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['obv'],
                             mode = 'lines',
                             line_width = 1,
                             name = 'OBV'),
                  row = 2, col = 1)

    fig.update_yaxes(title = "Stock Price", row = 1, col = 1, side = 'right')
    fig.update_yaxes(title = "On-Balance-Volume", row = 2, col = 1, side = 'right')
    fig.update_layout(title = ticker + ' Stock',
                     legend = dict(yanchor="top", y = 1,
                                   xanchor= "right", x = 1.35))

#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code 
    
    fig.show(renderer="svg")

In [ ]:
#Test for AAPL ticker
obv_fig('AAPL', periods = 21)

## Average Directional Index - ADX 

Source: https://www.investopedia.com/terms/a/adx.asp

The average directional index (ADX) is a technical analysis indicator used by some traders to determine the strength of a trend. The trend can be either up or down, and this is shown by two accompanying indicators, the Negative Directional Indicator (-DI) and the Positive Directional Indicator (+DI). Therefore, ADX commonly includes three separate lines. These are used to help assess whether a trade should be taken long or short, or if a trade should be taken at all.

Calculating the Average Directional Movement Index (ADX)

Step 1: Calculate +DM, -DM, and True Range (TR) for each period. 14 periods are typically used.

- Use +DM when Current High - Previous High > Previous Low - Current Low:  +DM = Current High - Previous High.
- Use -DM when Previous Low - Current Low > Current High - Previous High.: -DM = Previous Low - Current Low.
- TR is the greater of the Current High - Current Low, Current High - Previous Close, or Current Low - Previous Close.

Step 2: Smooth the 14-period averages of +DM, -DM, and TR. Steps to calculate the TR formula is below. Insert the -DM and +DM values to calculate the smoothed averages of those.

- First 14TR = Sum of first 14 TR readings.
- Next 14TR value = First 14TR - (Prior 14TR/14) + Current TR
- Next, divide the smoothed +DM value by the smoothed TR value to get +DI. Multiply by 100.
- Divide the smoothed -DM value by the smoothed TR value to get-DI. Multiply by 100.

Step 3: The Directional Movement Index (DX) is +DI minus -DI, divided by the sum of +DI and -DI (all absolute values). Multiply by 100.

Step 4: To get the ADX, continue to calculate DX values for at least 14 periods. Then, smoothe the results to get ADX

- First ADX = sum 14 periods of DX / 14
- After that, ADX = ((Prior ADX * 13) + Current DX) /14

What Does the Average Directional Index (ADX) Tell You? The Average Directional Index (ADX) along with the Negative Directional Indicator (-DI) and the Positive Directional Indicator (+DI) are momentum indicators. The ADX helps investors determine trend strength while -DI and +DI help determine trend direction.

The ADX identifies a strong trend when the ADX is over 25 and a weak trend when the ADX is below 20.

Crossovers of the -DI and +DI lines can be used to generate trade signals. For example: 
- If the +DI line crosses above the -DI line and the ADX is above 20, or ideally above 25, then that is a potential signal to buy.
- If the -DI crosses above the +DI, and ADX is above 20 or 25, then that is an opportunity to enter a potential short trade.
- Crosses can also be used to exit current trades. For example, if long, exit when the -DI crosses above the +DI.

When ADX is below 20 the indicator is signaling that the price is trendless, and therefore may not be an ideal time to enter a trade.

In [ ]:
#Compute average true range
def average_true_range(ticker, trend_periods=14): 
    data = info(ticker)
    data = data.reset_index(drop=False)
    atr = []
    for index, row in data.iterrows():
        prices = [row['High'], row['Low'], row['Close'], row['Open']]
        if index > 0:
            val1 = np.nanmax(prices) - np.nanmin(prices)
            val2 = abs(np.nanmax(prices) - data.at[index - 1, 'Close'])
            val3 = abs(np.nanmin(prices) - data.at[index - 1, 'Close'])
            true_range = np.nanmax([val1, val2, val3])

        else:
            true_range = np.nanmax(prices) - np.nanmin(prices)

        atr.append(true_range)
        result = pd.DataFrame({'true_range': atr})
    result['atr'] = result['true_range'].ewm(ignore_na=False, min_periods=0, com=trend_periods, adjust=True).mean()
        
    return result

In [ ]:
#Compute ADX index
def directional_movement_index(ticker, periods=14): 
    data = info(ticker)
    data = data.reset_index(drop=False)
    r = data.shape[0]
    result = pd.DataFrame(np.zeros((r, 9)))
    result.columns = ['true_range', 'm_plus', 'm_minus', 'dm_plus', 'dm_minus', 'di_plus', 'di_minus', 'dxi', 'adx']
    atr = average_true_range(ticker, trend_periods=14)
    result['true_range'] = atr['true_range']
    
    for i,row in data.iterrows():
        if i>0:
            result.at[i, 'm_plus'] = row['High'] - data.at[i-1, 'High']
            result.at[i, 'm_minus'] = row['Low'] - data.at[i-1, 'Low']
    
    for i,row in result.iterrows():
        if row['m_plus'] > row['m_minus'] and row['m_plus'] > 0:
            result.at[i, 'dm_plus'] = row['m_plus']
            
        if row['m_minus'] > row['m_plus'] and row['m_minus'] > 0:
            result.at[i, 'dm_minus'] = row['m_minus']
    
    result['di_plus'] = 100 * (result['dm_plus'] / result['true_range']).ewm(ignore_na=False, min_periods=0, com=periods, adjust=True).mean()
    result['di_minus'] = 100 * (result['dm_minus'] / result['true_range']).ewm(ignore_na=False, min_periods=0, com=periods, adjust=True).mean()
    
    result['dxi'] = np.abs(result['di_plus'] - result['di_minus']) / (result['di_plus'] + result['di_minus']) *100
    result.at[0, 'dxi'] = 1
    result['adx'] = result['dxi'].ewm(ignore_na=False, min_periods=0, com=periods, adjust=True).mean()
    result = result.drop(['true_range', 'm_plus', 'm_minus', 'dm_plus', 'dm_minus'], axis=1)
    data = pd.concat([data,result], axis=1)
    data = data.set_index('Date')
    
    return data

In [ ]:
#Trading action decision
def adx_action(ticker):    
    data = directional_movement_index(ticker, periods=14)
    data = data.reset_index(drop=False)
    r = data.shape[0]
    trend = pd.DataFrame(np.zeros((r, 3)))
    trend.columns = ['trend strength','trend direction','action']
    trend['trend strength'] = trend['trend strength'].astype(str)
    trend['trend direction'] = trend['trend direction'].astype(str)
    trend['action'] = trend['action'].astype(str)
    
    for i,row in data.iterrows():
        if row['adx'] < 25:
            trend.at[i, 'trend strength'] = 'weak'
        elif row['adx'] > 25 and row['adx'] < 50:
            trend.at[i, 'trend strength'] = 'strong'
        elif row['adx'] > 50 and row['adx'] < 75:
            trend.at[i, 'trend strength'] = 'very strong'
        else:
            trend.at[i, 'trend strength'] = 'extremely strong'

    trend.at[0, 'trend direction'] = 'NA'
    for i in range(1,r):
        if data.at[i, 'di_plus'] >= data.at[i, 'di_minus'] and data.at[i-1, 'di_plus'] <= data.at[i-1, 'di_minus']:
            trend.at[i, 'trend direction'] = 'buy'
        elif data.at[i, 'di_plus'] <= data.at[i, 'di_minus'] and data.at[i-1, 'di_plus'] >= data.at[i-1, 'di_minus']:
            trend.at[i, 'trend direction'] = 'sell'
        else:
            trend.at[i, 'trend direction'] = 'NA'

    for i,row in data.iterrows():
        if trend.at[i, 'trend direction'] == 'NA':
            trend.at[i, 'action'] = 'NA'
        else:
            trend.at[i, 'action'] = trend.at[i, 'trend strength'] + ' possible to ' + trend.at[i, 'trend direction']
    data = pd.concat([data,trend['trend direction'],trend['action']], axis=1)
    data = data.set_index('Date')
    
    return data

def adx_action_date(ticker, date):  
    data = adx_action(ticker)
    if (date in data.index):
        result = data.at[date, 'action']
    else:
        result = print ('Date not exist')
        
    return result
    

In [ ]:
#Code buy-sell signal for adx plot
def sell_buy_signal(ticker,periods): 
    data = adx_action(ticker)
    data = data.reset_index(drop=False)
    buys = pd.DataFrame(np.zeros((data.shape[0], 1)))
    buys.columns = ['buy']
    sells = pd.DataFrame(np.zeros((data.shape[0], 1)))
    sells.columns = ['sell']
    for i,row in data.iterrows():
            if (row['trend direction'] == 'buy'):
                buys.at[i, :] =  data.at[i, 'Close']
            else:
                buys.at[i, :] =  np.nan

    for i,row in data.iterrows():
            if (row['trend direction'] == 'sell'):
                sells.at[i, :] =  data.at[i, 'Close']
            else:
                sells.at[i, :] =  np.nan
    re = pd.concat([buys,sells], axis=1)
    
    return re

In [ ]:
#Code adx plot/graph/visual
def adx_fig(ticker, periods):
    data = directional_movement_index(ticker, periods)
    signal = sell_buy_signal(ticker,periods)
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.25)
    fig.add_trace(go.Candlestick(x = data.index,
                                 open = data['Open'],
                                 high = data['High'],
                                 low = data['Low'],
                                 close = data['Close'],
                                 name = 'Stock price'), 
                  row = 1, col = 1)

    avg_14_adj = data.Close.rolling(window=14, min_periods=1).mean()
    fig.add_trace(go.Scatter(x = data.index, 
                             y = avg_14_adj,
                             line = dict(color='blue', width=1, dash='dash'),
                             name = 'Close, 14-Day EMA'),
                  row = 1, col = 1)

    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['di_plus'],
                             line = dict(color = 'black', width=1),
                             name = '+DI'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['di_minus'],
                             line = dict(color = 'red', width=1),
                             name = '-DI'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['adx'],
                             line = dict(color = 'green', width=1),
                             name = 'ADX'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['Close'],
                             line = dict(color = 'blue', width=1),
                             name = 'Closing Price'),
                  row = 3, col = 1)
    fig.add_trace(go.Scatter(mode='markers',
                             x = data.index, 
                             y = signal['sell'],
                             marker=dict(symbol = 6, size=5, color = 'red'),
                             name = 'Sell'),
                  row = 3, col = 1)
    fig.add_trace(go.Scatter(mode='markers',
                             x = data.index, 
                             y = signal['buy'],
                             marker=dict(symbol = 5, size=5, color = 'green'),
                             name = 'Buy'),
                  row = 3, col = 1)
    fig.update_yaxes(title = "Stock Price", row = 1, col = 1, side = 'right')
    fig.update_yaxes(title = "ADX", row = 2, col = 1, side = 'right')
    fig.update_yaxes(title = "Trading Signal", row = 3, col = 1, side = 'right')
    fig.update_layout(title = ticker + ' Stock',
                     legend = dict(yanchor="top", y = 1,
                                   xanchor= "right", x = 1.35))
#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code 

    return fig.show()

In [ ]:
#Code combine full adx calculation + trading signal + graph
def adx(ticker, periods):
    average_true_range(ticker, periods)
    directional_movement_index(ticker,periods)
    result = adx_action(ticker)
    result = result[result['action'] != "NA"]
    sell_buy_signal(ticker, periods) 
    adx_fig(ticker, periods)
    return result

In [ ]:
#Test on AAPL ticker
adx('AAPL', periods = 14)

## Simple moving average (SMA) 

Source: https://www.investopedia.com/terms/m/movingaverage.asp

Moving average is a simple, technical analysis tool. Moving averages are usually calculated to identify the trend direction of a stock or to determine its support and resistance levels. It is a trend-following–or lagging–indicator because it is based on past prices.

The longer the time period for the moving average, the greater the lag. Moving averages are a totally customizable indicator, which means that an investor can freely choose whatever time frame they want when calculating an average. The most common time periods used in moving averages are 15, 20, 30, 50, 100, and 200 days. The shorter the time span used to create the average, the more sensitive it will be to price changes. The longer the time span, the less sensitive the average will be.

Simple Moving Average Strategy is based on the simple moving average (SMA). The logic of the strategy can be summarized by the following:

- when the price crosses the t-day SMA upwards — buy shares
- when the price crosses the t-day SMA downwards — sell all the shares 

The moving average uses t-1 prior days and the current day — the trading decision is for the next day.

In [ ]:
#Simple moving average calculation code
def sma(ticker, period): 
    data = info(ticker)
    data['sma' + str(period)] = data.Close.rolling(window=period, min_periods=1).mean()
    
    return data

In [ ]:
#Trading action
def sma_action(ticker, period):    
    data = sma(ticker, period)
    data = data.reset_index(drop=False)
    r = data.shape[0]
    action = pd.DataFrame(np.zeros((r, 1)))
    action.columns = ['action']
    action['action'] = action['action'].astype(str)    

    action.at[0, 'action'] = 'NA'
    for i in range(1,r):
        # Cross up
        if data.at[i, 'Close'] >= data.at[i, 'sma' + str(period)] and data.at[i-1, 'Close'] <= data.at[i-1, 'sma' + str(period)]:
            action.at[i, 'action'] = 'buy'
        # Cross down
        elif data.at[i, 'Close'] <= data.at[i, 'sma' + str(period)] and data.at[i-1, 'Close'] >= data.at[i-1, 'sma' + str(period)]:
            action.at[i, 'action'] = 'sell'
        else:
            action.at[i, 'action'] = 'NA'

    data = pd.concat([data,action['action']], axis=1)
    data = data.set_index('Date')
    
    return data

#Code seperate check for action on a specific day
def sma_action_date(ticker, date, short_period, long_period):  
    data = sma_action(ticker, short_period, long_period)
    if (date in data.index):
        result = data.at[date, 'action']
    else:
        result = print ('Date not exist')
        
    return result

In [ ]:
#Code sell-buy signal check
def sell_buy_signal(ticker, period):
    data = sma_action(ticker, period)
    data = data.reset_index(drop=False)
    buys = pd.DataFrame(np.zeros((data.shape[0], 1)))
    buys.columns = ['buy']
    sells = pd.DataFrame(np.zeros((data.shape[0], 1)))
    sells.columns = ['sell']
    for i,row in data.iterrows():
            if (row['action'] == 'buy'):
                buys.at[i, :] =  data.at[i, 'Close']
            else:
                buys.at[i, :] =  np.nan

    for i,row in data.iterrows():
            if (row['action'] == 'sell'):
                sells.at[i, :] =  data.at[i, 'Close']
            else:
                sells.at[i, :] =  np.nan
    re = pd.concat([buys,sells], axis=1)
    
    return re

#Code SMA visual
def sma_fig(ticker, period):
    data = sma_action(ticker, period)
    signal = sell_buy_signal(ticker, period)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.25)
    fig.add_trace(go.Candlestick(x = data.index,
                                 open = data['Open'],
                                 high = data['High'],
                                 low = data['Low'],
                                 close = data['Close'],
                                 name = 'Stock price'), 
                  row = 1, col = 1)

    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['Close'],
                             line = dict(color = 'blue', width=1),
                             name = 'Closing Price'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['sma' + str(period)],
                             line = dict(color = 'black', width=1),
                             name = 'SMA - ' + str(period) + 'days'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(mode='markers',
                             x = data.index, 
                             y = signal['sell'],
                             marker=dict(symbol = 6, size=5, color = 'red'),
                             name = 'Sell'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(mode='markers',
                             x = data.index, 
                             y = signal['buy'],
                             marker=dict(symbol = 5, size=5, color = 'green'),
                             name = 'Buy'),
                  row = 2, col = 1)
    fig.update_yaxes(title = "Stock Price", row = 1, col = 1, side = 'right')
    fig.update_yaxes(title = "Trading Signal", row = 3, col = 1, side = 'right')
    fig.update_layout(title = ticker + ' Stock',
                     legend = dict(yanchor="top", y = 1,
                                   xanchor= "right", x = 1.35))
#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code 

    return fig.show()

In [ ]:
#Test AAPL ticker
sma_fig('AAPL', period = 20)
data = sma_action('AAPL', 20)
data= data[data['action'] != "NA"]
data

## Moving Average Convergence Divergence – MACD 

Source: https://www.investopedia.com/terms/m/macd.asp

Moving Average Convergence Divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. The MACD is calculated by subtracting the 26-period Exponential Moving Average (EMA) from the 12-period EMA.

The result of that calculation is the MACD line. A nine-day EMA of the MACD called the "signal line," is then plotted on top of the MACD line, which can function as a trigger for buy and sell signals. Traders may buy the security when the MACD crosses above its signal line and sell - or short - the security when the MACD crosses below the signal line. Moving Average Convergence Divergence (MACD) indicators can be interpreted in several ways, but the more common methods are crossovers, divergences, and rapid rises/falls.

The Formula for MACD Is: MACD = 12-Period EMA - 26-Period EMA

A nine-day EMA of the MACD called the "signal line," is then plotted on top of the MACD line, which can function as a trigger for buy and sell signals. Traders may buy the security when the MACD crosses above its signal line and sell - or short - the security when the MACD crosses below the signal line. Moving Average Convergence Divergence (MACD) indicators can be interpreted in several ways, but the more common methods are crossovers, divergences, and rapid rises/falls.

The strategy can be described by:

buy shares when the MACD crosses the signal line upwards
sell shares when the MACD crosse s the signal line downwards
MACD is often displayed with a histogram which graphs the distance between the MACD and its signal line. If the MACD is above the signal line, the histogram will be above the MACD’s baseline. If the MACD is below its signal line, the histogram will be below the MACD’s baseline. Traders use the MACD’s histogram to identify when bullish or bearish momentum is high.

![image](https://www.investopedia.com/thmb/4oNhG4hraF3Lwe9es10gUindpTY=/6146x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/dotdash_Final_Moving_Average_Convergence_Divergence_MACD_Aug_2020-01-40afb99e03974737802ddd750b97cdaa.jpg)

In [ ]:
#Code MACD calculate
def macd(ticker):
    data = info(ticker)
    period_long = 26
    period_short = 12
    period_signal = 9
    remove_cols = []
    data['ema' + str(period_long)] = data['Close'].ewm(ignore_na=False, min_periods=period_long, com=period_long, adjust=True).mean()
    remove_cols.append('ema' + str(period_long))
    data['ema' + str(period_short)] = data['Close'].ewm(ignore_na=False, min_periods=period_short, com=period_short, adjust=True).mean()
    remove_cols.append('ema' + str(period_short))
    data['macd'] = data['ema' + str(period_short)] - data['ema' + str(period_long)]
    data['signal_line'] = data['macd'].ewm(ignore_na=False, min_periods=0, com=period_signal, adjust=True).mean()
    data = data.drop(remove_cols, axis=1)
    
    return data

In [ ]:
#Trading action
def macd_action(ticker):    
    data = macd(ticker)
    data = data.reset_index(drop=False)
    r = data.shape[0]
    action = pd.DataFrame(np.zeros((r, 1)))
    action.columns = ['action']
    action['action'] = action['action'].astype(str)    

    action.at[0, 'action'] = 'NA'
    for i in range(1,r):
        # Cross up
        if data.at[i, 'macd'] >= data.at[i, 'signal_line'] and data.at[i-1, 'macd'] <= data.at[i-1, 'signal_line']:
            action.at[i, 'action'] = 'buy'
        # Cross down
        elif data.at[i, 'macd'] <= data.at[i, 'signal_line'] and data.at[i-1, 'macd'] >= data.at[i-1, 'signal_line']:
            action.at[i, 'action'] = 'sell'
        else:
            action.at[i, 'action'] = 'NA'

    data = pd.concat([data,action['action']], axis=1)
    data = data.set_index('Date')
    
    return data

#Code single check for trading action on a single date
def macd_action_date(ticker, date):  
    data = macd_action(ticker)
    if (date in data.index):
        result = data.at[date, 'action']
    else:
        result = print ('Date not exist')
        
    return result

In [ ]:
#Code MACD visual
def macd_fig(ticker):
    data1 = macd_action("AAPL")
    data = macd(ticker)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.25)
    fig.add_trace(go.Candlestick(x = data.index,
                                 open = data['Open'],
                                 high = data['High'],
                                 low = data['Low'],
                                 close = data['Close'],
                                 name = 'Stock price'), 
                  row = 1, col = 1)

    avg_14_adj = data.Close.rolling(window=14, min_periods=1).mean()
    fig.add_trace(go.Scatter(x = data.index, 
                             y = avg_14_adj,
                             line = dict(color='blue', width=1),
                             name = 'Close, 14-Day EMA'),
                  row = 1, col = 1)

    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['signal_line'],
                             line = dict(color = 'orange', width=1),
                             name = 'Signal line'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(x = data.index, 
                             y = data['macd'],
                             line = dict(color = 'green', width=1),
                             name = 'MACD'),
                  row = 2, col = 1)
    fig.add_trace(go.Scatter(x = data.index, 
                             y = np.zeros(data.shape[0]),
                             line = dict(color = 'red', width=1, dash='dash'),
                             name = 'Basic Line'),
                  row = 2, col = 1)
#     fig.add_trace(go.Bar(x = data.index, 
#                              y = data['macd'] - data['signal_line'],
#                              marker_color ='red',
#                              name = 'Signal Trading'), #Lỗi chỉnh mở signal
#                   row = 2, col = 1)

    fig.add_trace(go.Scatter(x=data1[data1['action'] == 'sell'].index, 
                             y=data1['macd'][data1['action'] == 'sell'],
                             mode='markers',
                             marker=dict(symbol = 6, size=5, color = 'red'),
                             name='Sell signal'),
                  row = 2, col = 1)#sell signal

    #Plot Buy signal and Divvergence reason
    fig.add_trace(go.Scatter(x=data1[data1['action'] == 'buy'].index, 
                             y=data1['macd'][data1['action'] == 'buy'],
                             mode='markers',
                             marker=dict(symbol = 5, size=5, color = 'green'),
                             name='Buy signal'),
                  row = 2, col = 1)#Buy signal
    
    fig.update_yaxes(title = "Stock Price", row = 1, col = 1, side = 'right')
    fig.update_yaxes(title = "MACD", row = 2, col = 1, side = 'right')
    fig.update_layout(title=ticker + ' Stock',
    #                   showlegend=False,
                      autosize=True, width=950,height=800,
                      shapes=[dict(type="rect",
                                   xref="x1",# x-reference is assigned to the x-values
                                   yref="y2",  # y-reference is assigned to the plot paper [0,1]
                                   x0="2014-12-31", y0=20,
                                   x1=date.today(), y1=15,
                                   fillcolor="rgba(231,107,243,0.2)",
                                   opacity=0.5,
                                   layer="below",
                                   line_width=0,)],
                      legend = dict(yanchor="top",y = 1,
                                    xanchor= "right", x = 1.35))
#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code     
    
    return fig.show()

In [ ]:
#Test MACD visual
macd_fig('AAPL')

data1 = macd_action("AAPL")
data1 = data1[data1['action'] != 'NA']
data1

## Stochastic Oscillator 

Source: https://www.investopedia.com/terms/s/stochasticoscillator.asp

The formula for the stochastic oscillator is:
$$\%K = \frac{C - L14}{H14 - L14} * 100$$
- C = The most recent closing price
- L14 = The lowest price traded of the 14 previous trading sessions
- H14 = The highest price traded of the 14 previous trading sessions
- %K = The current value of the stochastic oscillator

%K is referred to sometimes as the slow stochastic indicator. The "fast" stochastic indicator is taken as %D = 3-period moving average of %K.

The stochastic oscillator is range-bound, meaning it is always between 0 and 100. This makes it a useful indicator of overbought and oversold conditions. Traditionally, readings over 80 are considered in the overbought range, and readings under 20 are considered oversold. However, these are not always indicative of impending reversal; very strong trends can maintain overbought or oversold conditions for an extended period. Instead, traders should look to changes in the stochastic oscillator (fast SO or 3-period moving average of %K) for clues about future trend shifts.

Stochastic oscillator charting generally consists of two lines: one reflecting the actual value of the oscillator for each session, and one reflecting its three-day simple moving average. Because price is thought to follow momentum, intersection of these two lines is considered to be a signal that a reversal may be in the works, as it indicates a large shift in momentum from day to day.

Divergence between the stochastic oscillator and trending price action is also seen as an important reversal signal. For example, when a bearish trend reaches a new lower low, but the oscillator prints a higher low, it may be an indicator that bears are exhausting their momentum and a bullish reversal is brewing.

In [ ]:
#trade strategy: against market trend, "be fearful when others are greedy, and greedy when others are fearful"

#Code SO visual
def graph_SO(app, ticker):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.23)
#     d = date.today()

    #Plot candle stick for stock price
    fig.add_trace(go.Candlestick(x=app.index,
                                 open=app['Open'],
                                 high=app['High'],
                                 low=app['Low'],
                                 close=app['Close'],
                                 name = 'Stock price'), 
                  row = 1, col = 1)
    fig.update_yaxes(title= "Stock Price", row=1, col=1, side='right')

    #Plot Stochastic Oscillator (%K line) and fast Stochastic Oscillator or 3 days MA of %K line (%D line)
    fig.add_trace(go.Scatter(x=app.index, 
                             y=app['%K'],
                             mode='lines',
                             line_width=1,
                             line_color='royalblue',
                             name='%K'),
                  row = 2, col = 1)

    fig.add_trace(go.Scatter(x=app.index, 
                             y=app['%D'],
                             line = dict(color='red', width=1, dash='dash'),
                             name='%D'),
                  row = 2, col = 1)

    fig.update_yaxes(title_text="Stochastic Oscillator", range=[-15, 115], row=2, col=1, side='right')

    #Plot oscallators cross points
    fig.add_trace(go.Bar(x = app.index,
                         y = app['KvsD'][app['KvsD']>=0],
                         marker_color = 'green',
                         name = '%K over %D'),
                  row = 2, col = 1)
    fig.add_trace(go.Bar(x = app.index,
                         y = app['KvsD'][app['KvsD']<0],
                         marker_color = 'red',
                        name = '%K under %D'),
                  row = 2, col = 1)

    #Plot Sell signal and Divvergence reason

    fig.add_trace(go.Scatter(x=app[app['R'] == 'Sell'].index, 
                             y=app['%D'][app['R'] == 'Sell'],
                             mode='markers',
                             marker=dict(symbol = 6, size=5, color = 'red'),
                             name='Sell signal'),
                  row = 2, col = 1)#sell signal
#     h = app[app['R'] == 'Sell'].index
#     for i in h:
#         x0 = app.loc[:i].index[-14]
#         x1 = i
#         fig.add_trace(go.Scatter(x=[x0,x1], 
#                                  y=app['High'].loc[[x0,x1]],
#                                  mode='lines',
#                                  line = dict(color='red', width=1),
#                                  name='Market bear sign'),
#                       row = 1, col = 1)

    #Plot Buy signal and Divvergence reason
    fig.add_trace(go.Scatter(x=app[app['R'] == 'Buy'].index, 
                             y=app['%D'][app['R'] == 'Buy'],
                             mode='markers',
                             marker=dict(symbol = 5, size=5, color = 'green'),
                             name='Buy signal'),
                  row = 2, col = 1)

#     h = app[app['R'] == 'Buy'].index
#     for i in h:
#         x0 = app.loc[:i].index[-14]
#         x1 = i
#         fig.add_trace(go.Scatter(x=[x0,x1], 
#                                  y=app['Low'].loc[[x0,x1]],
#                                  mode='lines',
#                                  line = dict(color='green', width=1),
#                                  name='Market bull sign'),
#                       row = 1, col = 1)

    #Layout 

    fig.update_layout(title= ticker + ' Stock',
    #                   showlegend=False,
                      autosize=False, width=950,height=800,
                      shapes=[dict(type="rect",
                                   xref="x1",# x-reference is assigned to the x-values
                                   yref="y2",  # y-reference is assigned to the plot paper [0,1]
                                   x0="2014-12-31", y0=20,
                                   x1=date.today(), y1=80,
                                   fillcolor="rgba(231,107,243,0.2)",
                                   opacity=0.5,
                                   layer="below",
                                   line_width=0,)],
                      legend = dict(yanchor="top",y = 1,
                                    xanchor= "right", x = 1.3))

    # fig.add_annotation(x=d,y=app.loc[d,'Close'],
    #                    xref="x1",yref="y1",
    #                    text=app.loc[d,'Close'].round(1),
    #                    align="right",
    #                    bgcolor="#ff7f0e",
    #                    opacity=0.8)
    
#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code     
    
    return fig.show()

#Code SO calculation    
def SO(ticker): 
    data = info(ticker)
    p = 14
    for d in data.index[15:]:
        #Calculate oscallator
        d_ss = d - timedelta(days=p)
        c = data['Close'].loc[d]
        M_L14 = data['Low'].loc[d_ss:].min()
        M_H14 = data['High'].loc[d_ss:].max()
        so = ((c-M_L14)/(M_H14-M_L14))*100
        data.loc[d,'%K'] = so
        data['%D'] = data['%K'].rolling(window=3).mean()
        
        #Check overbought or oversold signal from oscallator (round1 assess)
        if data.loc[d,'%D'] >= 80:
            data.loc[d,'SO_market'] = 'over_bought'
        elif data.loc[d,'%D'] <= 20:
            data.loc[d,'SO_market'] = 'over_sold'
        
        #Check Divergence Vs. Oscillators (final assess)
        I_H14 = data['%D'].loc[d_ss:].max()
        I_L14 = data['%D'].loc[d_ss:].min()
        if data.loc[d,'High'] > M_H14 and data.loc[d,'%D'] < I_H14:
            data.loc[d,'Div_M'] = 'A bear'#strong market down
        elif data.loc[d,'High'] == M_H14 and data.loc[d,'%D'] < I_H14:
            data.loc[d,'Div_M'] = 'B bear'#medium market down
        elif data.loc[d,'High'] < M_H14 and data.loc[d,'%D'] == I_H14:
            data.loc[d,'Div_M'] = 'C bear' #weak medium market down   
        elif data.loc[d,'High'] < M_H14 and data.loc[d,'%D'] > I_H14:
            data.loc[d,'Div_M'] = 'H bear'  #hidden market down  
        elif data.loc[d,'Low'] < M_L14 and data.loc[d,'%D'] > I_L14:
            data.loc[d,'Div_M'] = 'A bull'#strong market up
        elif data.loc[d,'Low'] == M_L14 and data.loc[d,'%D'] > I_L14:
            data.loc[d,'Div_M'] = 'B bull'#medium market up
        elif data.loc[d,'Low'] < M_L14 and data.loc[d,'%D'] == I_L14:
            data.loc[d,'Div_M'] = 'C bull'#weak market up
        elif data.loc[d,'Low'] < M_L14 and data.loc[d,'%D'] < I_L14:
            data.loc[d,'Div_M'] = 'H bull' #hidden market up
            
    #Check oscallators crossover for potential market up or down (round2 assess)
    data['KvsD'] = data['%K'] - data['%D']
    i = data.index
    n = len(i)
    for j in range(1,n):
        if data.loc[i[j-1],'KvsD'] <=0 and data.loc[i[j],'KvsD']>0:
            data.loc[i[j],'Div_O']= 'bull'
        elif data.loc[i[j-1],'KvsD'] >0 and data.loc[i[j],'KvsD']<0:
            data.loc[i[j],'Div_O']= 'bear'
    
    #Combine assess conds for conclusion
    data['R'] = np.nan
    
    #case 1: both %D and %K over 50 for sell (standard80) and under 20 for buy + oscallator momentum move the same way
    cond1_buy = (data['%K'] >=50) & (data['%D'] >=50) & (data['Div_O'] == 'bull')     
    for d in data[cond1_buy].index:
        data.loc[d,'R'] = 'Sell'
    cond1_sell = (data['%K'] <=20) & (data['%D'] <=20) & (data['Div_O'] == 'bear')      
    for d in data[cond1_sell].index:
        data.loc[d,'R'] = 'Buy'    
        
    #case2: market reversal by divergence between the stochastic oscillator and trending price action
    cond2_buy = data['Div_M'].str.match(r'[AB] bull') == True
    cond3_buy = data['Div_O'].str.match(r'bear') == True
    buy = cond2_buy & cond3_buy
    for d in data[buy].index:
        data.loc[d,'R'] = 'Sell'
    
    cond2_sell = data['Div_M'].str.match(r'[AB] bear') == True
    cond3_sell = data['Div_O'].str.match(r'bull') == True
    sell = cond2_sell & cond3_sell
    for d in data[sell].index:
        data.loc[d,'R'] = 'Buy'

    data_r = data['R']#result table
    
    graph_SO(data, ticker)
    
    return data , data_r
    
#Test with AAPL ticker    
app,app_r = SO('AAPL')
app[app['R'].isnull() == False]

## Bollinger Band 

Source: https://www.investopedia.com/terms/b/bollingerbands.asp

A Bollinger Band® is a technical analysis tool defined by a set of trendlines plotted two standard deviations (positively and negatively) away from a simple moving average (SMA) of a security's price, but which can be adjusted to user preferences.

Bollinger Bands® are a highly popular technique. Many traders believe the closer the prices move to the upper band, the more overbought the market, and the closer the prices move to the lower band, the more oversold the market.

In the chart depicted below, Bollinger Bands® bracket the 20-day SMA of the stock with an upper and lower band along with the daily movements of the stock's price. Because standard deviation is a measure of volatility, when the markets become more volatile the bands widen; during less volatile periods, the bands contract.

![image](https://www.investopedia.com/thmb/gvlXmWskHGDZvNuPRNQdkxO-DY0=/1543x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/BollingerBands-5c535dc646e0fb00013a1b8b.png)

Bollinger Band formular:
$$BOLU = MA(TP,n) + m*\sigma[TP,n]$$

$$BOLD = MA(TP,n) - m*\sigma[TP,n]$$

- BOLU = Upper Bollinger band
- BOLD = Lower Bollinger band
- MA = Moving average
- TP (typical price) = (High + Low + Close)/3
- n = Number of days in smoothing period (typically 20)
- m = Number of standard deviations (typically 2)
- $\sigma[TP,n]$ = Standard Deviations over last n periods of TP

In [ ]:
##trade strategy: against market trend, "be fearful when others are greedy, and greedy when others are fearful"

#Code BB visual
def graph_BB(app,ticker):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    fig = go.Figure()
    # fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.23)

    #Plot candle stick for stock price
    fig.add_trace(go.Candlestick(x=app.index,
                                 open=app['Open'],
                                 high=app['High'],
                                 low=app['Low'],
                                 close=app['Close'],
                                 name = 'Stock price'))

    fig.update_yaxes(title= "Stock price", side='right')

    #Plot Bollinger Bands
    fig.add_trace(go.Scatter(x=app.index,
                             y=app['BOLU'],
                             fill='none',
                             fillcolor='rgba(231,107,243,0.2)',
                             line_color='rgba(255,255,255,0)',
                             showlegend=False,
                             name='Bollinger Band',
                             ))
    fig.add_trace(go.Scatter(x=app.index,
                             y=app['BOLD'],
                             fill='tonexty',
                             fillcolor='rgba(231,107,243,0.2)',
                             line_color='rgba(255,255,255,0)',
                             showlegend=True,
                             name='Bollinger Band',
                             ))


    fig.add_trace(go.Scatter(x=app.index, 
                             y=app['MA20-TP'],
                             line_color='rgb(231,107,243)',
                             name='MA 20 - Typical Price',
                             ))

    #Plot Sell signal
    fig.add_trace(go.Scatter(x=app[app['R'] == 'Sell'].index, 
                                     y=app['High'][app['R'] == 'Sell'],
                                     mode='markers',
                                     marker=dict(symbol = 6, size=10, color = 'red'),
                                     name='Sell signal'))
    #Plot Buy signal
    fig.add_trace(go.Scatter(x=app[app['R'] == 'Buy'].index, 
                                     y=app['Low'][app['R'] == 'Buy'],
                                     mode='markers',
                                     marker=dict(symbol = 5, size=10, color = 'green'),
                                     name='Buy signal'))

    #Layout 
    fig.update_layout(title= ticker + ' Stock',
    #                   showlegend=False,
                      autosize=False, width=950,height=800,
                      legend = dict(yanchor="top",y = 1,
                                    xanchor= "right", x = 1.3))

    # fig.add_annotation(x=d,y=app.loc[d,'Close'],
    #                    xref="x",yref="y",
    #                    text=app.loc[d,'Close'].round(1),
    #                    align="right",
    #                    bgcolor="#ff7f0e",
    #                    opacity=0.8)
    
#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code     
    
    return fig.show()

#Code BB calculation    
def BB(ticker):
    n = 20
    data = info(ticker)
    data['TP'] = (data['High']+data['Low']+data['Close'])/3
    data['MA20-TP'] = data['TP'].rolling(window=n).mean()
    index = data.index
    for i in range(n,len(index)):
        ds = index[i-n]
        d = index[i]
        data1= data.loc[ds:d,'TP']
        data.loc[d,'Std'] = data1.std()
    
    #Calculate upper and lower band
    data['BOLU'] = data['MA20-TP'] + 2*data['Std']
    data['BOLD'] = data['MA20-TP'] - 2*data['Std']
    data['R'] = np.nan
    
    #Identity market trend and decision
    index = data.index
    p = 0.6 #if p increase then target_price up and down move closer to MA20 line
    for i in range(n,len(index)):
        ds = index[i-n]
        d = index[i]
        df = data.loc[ds:d,:]
        M_L20 = df['Low'].min()
        M_H20 = df['High'].max()
        C_L = data.loc[d,'Low']
        C_H = data.loc[d,'High']
        target_price_up = data.loc[d,'BOLU']-2*p*data.loc[d,'Std']
        target_price_down = data.loc[d,'BOLD']+2*p*data.loc[d,'Std']
        
        if C_L <= M_L20 and C_H <= M_H20: #condition identify market down trend
            data.loc[d,'M_trend'] = 'bear'
#             data.loc[d,'R'] = 'Sell'
            if data.loc[d,'High']>= target_price_up:#sell when price is near up band
                data.loc[d,'R'] = 'Sell'
        
        if C_L >= M_L20 and C_H >= M_H20: #condition identify market up trend
            data.loc[d,'M_trend'] = 'bull'
#             data.loc[d,'R'] = 'Buy'
            if data.loc[d,'Low']<= target_price_down:#buy when price is near down band
                data.loc[d,'R'] = 'Buy'
    
    
    graph_BB(data, ticker)
    data_r = data['R']
    return data, data_r

#Test BB with AAPL ticker
app, app_r = BB(ticker = 'AAPL')        
app[app['R'].isnull() == False]   

## Money Flow Index (MFI)

Source: https://www.investopedia.com/terms/m/mfi.asp

This method only show market trend.

The Money Flow Index (MFI) is a technical indicator that generates overbought or oversold signals using both prices and volume data.

An MFI reading above 80 is considered overbought and an MFI reading below 20 is considered oversold, although levels of 90 and 10 are also used as thresholds.

A divergence between the indicator and price is noteworthy. For example, if the indicator is rising while the price is falling or flat, the price could start rising.

$$Money Flow Index (MFI) = 100 - \frac{100}{1+ Money Flow Ratio}$$

- Money Flow Ratio = $\frac{14 Period Positive Money Flow}{14 Period Negative Money Flow}$

- Raw Money Flow = Typical Price * Volumn
- Typical Price = $\frac {High + Low + Close}{3}$

In [ ]:
#Code MFI visual
def graph_MFI(app, ticker):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.23)

    #Plot candle stick for stock price
    fig.add_trace(go.Candlestick(x=app.index,
                                 open=app['Open'],
                                 high=app['High'],
                                 low=app['Low'],
                                 close=app['Close'],
                                 name = 'Stock price'), 
                  row = 1, col = 1)
    fig.update_yaxes(title="Stock Price", row=1, col=1, side='right')

    #Plot MFI
    fig.add_trace(go.Scatter(x=app.index, 
                             y=app['MFI'],
                             mode='lines',
                             line_width=1,
                             name='MFI'),
                  row = 2, col = 1)

    fig.update_yaxes(title_text="MFI",
                     range=[-10, 110], 
                     row=2, col=1, side='right')

    #Layout 

    fig.update_layout(title=ticker + ' Stock',
#                       showlegend=False,
    #                   autosize=False, width=950,height=800,
                      shapes=[dict(type="rect",
                                   xref="x1",# x-reference is assigned to the x-values
                                   yref="y2",  # y-reference is assigned to the plot paper [0,1]
                                   x0="2014-12-31", y0=10,
                                   x1=date.today(), y1=20,
                                   fillcolor="salmon",
                                   opacity=0.5,
                                   layer="below",
                                   line_width=0,),
                              dict(type="rect",
                                   xref="x1",# x-reference is assigned to the x-values
                                   yref="y2",  # y-reference is assigned to the plot paper [0,1]
                                   x0="2014-12-31", y0=80,
                                   x1=date.today(), y1=90,
                                   fillcolor="salmon",
                                   opacity=0.5,
                                   layer="below",
                                   line_width=0,),
                              dict(type="rect",
                                   xref="x1",# x-reference is assigned to the x-values
                                   yref="y2",  # y-reference is assigned to the plot paper [0,1]
                                   x0="2014-12-31", y0=20,
                                   x1=date.today(), y1=80,
                                   fillcolor="rgba(231,107,243,0.2)",
                                   opacity=0.5,
                                   layer="below",
                                   line_width=0,)],
                     legend = dict(yanchor="top",y = 1,
                                   xanchor= "right", x = 1.3))

    d = date.today() - timedelta(days=1)

    # fig.add_annotation(x=d,y=app.loc[d,'Close'],
    #                    xref="x1",yref="y1",
    #                    text=app.loc[d,'Close'].round(1),
    #                    align="right",
    #                    bgcolor="#ff7f0e",
    #                    opacity=0.8)
    
#     py.plot(fig, filename = 'Quick_Stocks_Analysis_Sample', auto_open=True) #To edit in Plotly.com without real code     
    
    return fig.show()

#Code MFI calculation
def MFI(ticker):
    data = info(ticker)
    data['TP'] = (data['High'] + data['Low'] + data['Close'])/3
    
    index = data.index
    for i in range(1,len(index)):
        d = index[i]
        a = data.loc[:d,'TP']
        if a[-2] >= a[-1]:
            data.loc[d,'RMF_s'] = -1
        elif a[-2] < a[-1]:
            data.loc[d,'RMF_s'] = 1
        data.loc[d,'RMF'] = data.loc[d,'TP'] * data.loc[d,'Volume']*data.loc[d,'RMF_s']

    for i in range(15,len(index)):
        d14 = index[i-15]
        d = index[i]
        P = []
        N = []
        for v in data.loc[d14:d,'RMF']:
            if v >= 0:
                P.append(v)
            elif v!= np.nan and v<0:
                N.append(v)
        MFR = abs((sum(P))/(sum(N)))
        MFI = 100-(100/(1+MFR))
        data.loc[d,'MFI'] = MFI
        if data.loc[d,'MFI'] >= 80:
            data.loc[d,'M_trend'] = 'overbought'
        elif data.loc[d,'MFI'] <= 20:
            data.loc[d,'M_trend'] = 'oversold'
            
    graph_MFI(data, ticker)
    
    return data
    
#Test MFI with AAPL ticker
app = MFI('AAPL')
app = app[app['M_trend'].isnull() == False]
app